# **A study about the use of Graphs applied to a Transport Problem**
**This notebook is a form to practice my knowledge in data science**

The notebook walks us through a workflow for solving a problem with a tracking algorithm.

The main purpose of this notebook is to serve as a step-by-step workflow guide, allowing me to review this notebook myself and serve as a study for future cases.

## Workflow stages
The solution workflow goes through five stages.
1.   Load the Data.
2.   Define and create the graph.
3.   Defines direct relationships.
4.   Defines inverse relationships.
5.   Explore the graph.

In [0]:
#Import the library that creates the spark section
from pyspark.sql import SparkSession

In [0]:
#Starts the section for using spark
spark = SparkSession.builder.appName("transportGraphs").getOrCreate()

In [0]:
%fs ls /FileStore/tables

path,name,size,modificationTime
dbfs:/FileStore/tables/iris_bezdekIris.csv,iris_bezdekIris.csv,4551,1663856964000
dbfs:/FileStore/tables/movies-1.csv,movies-1.csv,494431,1662647401000
dbfs:/FileStore/tables/movies-2.csv,movies-2.csv,494431,1662647447000
dbfs:/FileStore/tables/movies.csv,movies.csv,494431,1662647363000
dbfs:/FileStore/tables/ratings.csv,ratings.csv,2483723,1662647649000
dbfs:/FileStore/tables/transport_nodes_7c826.csv,transport_nodes_7c826.csv,465,1666295023000
dbfs:/FileStore/tables/transport_relationships_c2bfc.csv,transport_relationships_c2bfc.csv,550,1666295023000
dbfs:/FileStore/tables/u.data,u.data,1979173,1662474869000


In [0]:
#Get the directory containing the nodes to used
nodesTransport="/FileStore/tables/transport_nodes_7c826.csv"

In [0]:
#Get the directory containing the file to use
relationshipsTransport="/FileStore/tables/transport_relationships_c2bfc.csv"

#1) Load the Data

In [0]:
#Reading stored files through generic function
dfNodes = spark.read.format('csv').options(header='true',delimiter=',', inferSchema=True).load(nodesTransport)

In [0]:
dfNodes.show()

+----------------+---------+---------+----------+
|              id| latitude|longitude|population|
+----------------+---------+---------+----------+
|       Amsterdam|52.379189| 4.899431|    821752|
|         Utrecht|52.092876|  5.10448|    334176|
|        Den Haag|52.078663| 4.288788|    514861|
|       Immingham| 53.61239| -0.22219|      9642|
|       Doncaster| 53.52285| -1.13116|    302400|
|Hoek van Holland|  51.9775|  4.13333|      9382|
|      Felixstowe| 51.96375|   1.3511|     23689|
|         Ipswich| 52.05917|  1.15545|    133384|
|      Colchester| 51.88921|  0.90421|    104390|
|          London|51.509865|-0.118092|   8787892|
|       Rotterdam|  51.9225|  4.47917|    623652|
|           Gouda| 52.01667|  4.70833|     70939|
+----------------+---------+---------+----------+



In [0]:
dfRelationships = spark.read.format('csv').options(header='true',delimiter=',', inferSchema=True).load(relationshipsTransport)

In [0]:
dfRelationships.show()

+----------------+----------------+------------+----+
|             src|             dst|relationship|cost|
+----------------+----------------+------------+----+
|       Amsterdam|         Utrecht|       EROAD|  46|
|       Amsterdam|        Den Haag|       EROAD|  59|
|        Den Haag|       Rotterdam|       EROAD|  26|
|       Amsterdam|       Immingham|       EROAD| 369|
|       Immingham|       Doncaster|       EROAD|  74|
|       Doncaster|          London|       EROAD| 277|
|Hoek van Holland|        Den Haag|       EROAD|  27|
|      Felixstowe|Hoek van Holland|       EROAD| 207|
|         Ipswich|      Felixstowe|       EROAD|  22|
|      Colchester|         Ipswich|       EROAD|  32|
|          London|      Colchester|       EROAD| 106|
|           Gouda|       Rotterdam|       EROAD|  25|
|           Gouda|         Utrecht|       EROAD|  35|
|        Den Haag|           Gouda|       EROAD|  32|
|Hoek van Holland|       Rotterdam|       EROAD|  33|
+----------------+----------

#2) Define and create the graph

In [0]:
from pyspark.sql.types import *
from graphframes import *  #Librarie for graph processing

In [0]:
#Defines the "Scheme" for each of the nodes
atributosNo = [
StructField("id", StringType(), True),\
StructField("latitude", FloatType(), True),\
StructField("longitude", FloatType(), True),\
StructField("population", IntegerType(), True)\
]

In [0]:
#Import the data in nodes
node = spark.read.csv(nodesTransport, header=True,schema=StructType(atributosNo))

#3) Defines direct relationships

In [0]:
#Import the data as relationships
direct_relationship = spark.read.csv(relationshipsTransport, header=True)

In [0]:
direct_relationship.show(5)

+---------+---------+------------+----+
|      src|      dst|relationship|cost|
+---------+---------+------------+----+
|Amsterdam|  Utrecht|       EROAD|  46|
|Amsterdam| Den Haag|       EROAD|  59|
| Den Haag|Rotterdam|       EROAD|  26|
|Amsterdam|Immingham|       EROAD| 369|
|Immingham|Doncaster|       EROAD|  74|
+---------+---------+------------+----+
only showing top 5 rows



#4) Defines inverse relationships

In [0]:
#Defines relationships by exchanging origins and destinations
inverse_relationship = (direct_relationship.withColumn("newSrc", direct_relationship.dst).withColumn("newDst", direct_relationship.src).drop("dst", "src").withColumnRenamed("newSrc", "src").withColumnRenamed("newDst", "dst").select("src", "dst", "relationship", "cost"))

In [0]:
inverse_relationship.show(5)

+---------+---------+------------+----+
|      src|      dst|relationship|cost|
+---------+---------+------------+----+
|  Utrecht|Amsterdam|       EROAD|  46|
| Den Haag|Amsterdam|       EROAD|  59|
|Rotterdam| Den Haag|       EROAD|  26|
|Immingham|Amsterdam|       EROAD| 369|
|Doncaster|Immingham|       EROAD|  74|
+---------+---------+------------+----+
only showing top 5 rows



In [0]:
#Create df with direct and inverse relationships (make the graph bidirectional)
relationships=direct_relationship.union(inverse_relationship)

In [0]:
relationships.show()

+----------------+----------------+------------+----+
|             src|             dst|relationship|cost|
+----------------+----------------+------------+----+
|       Amsterdam|         Utrecht|       EROAD|  46|
|       Amsterdam|        Den Haag|       EROAD|  59|
|        Den Haag|       Rotterdam|       EROAD|  26|
|       Amsterdam|       Immingham|       EROAD| 369|
|       Immingham|       Doncaster|       EROAD|  74|
|       Doncaster|          London|       EROAD| 277|
|Hoek van Holland|        Den Haag|       EROAD|  27|
|      Felixstowe|Hoek van Holland|       EROAD| 207|
|         Ipswich|      Felixstowe|       EROAD|  22|
|      Colchester|         Ipswich|       EROAD|  32|
|          London|      Colchester|       EROAD| 106|
|           Gouda|       Rotterdam|       EROAD|  25|
|           Gouda|         Utrecht|       EROAD|  35|
|        Den Haag|           Gouda|       EROAD|  32|
|Hoek van Holland|       Rotterdam|       EROAD|  33|
|         Utrecht|       Ams

In [0]:
#Define the graph 
graph=GraphFrame(node, relationships)

#5) Explore the graph

In [0]:
#Finding which are the cities with more than 100000 inhabitants and less than 300000
popMedia=graph.vertices\
.filter("population > 100000 and population < 300000")\
.sort("population")\
.show()

+----------+--------+---------+----------+
|        id|latitude|longitude|population|
+----------+--------+---------+----------+
|Colchester|51.88921|  0.90421|    104390|
|   Ipswich|52.05917|  1.15545|    133384|
+----------+--------+---------+----------+



In [0]:
#Showing the amount of direct paths (arriving)
display(graph.inDegrees)

id,inDegree
Doncaster,2
Rotterdam,3
London,2
Den Haag,4
Immingham,2
Colchester,2
Utrecht,2
Gouda,3
Ipswich,2
Hoek van Holland,3


In [0]:
#Showing the amount of inverse paths (outgoing)
display(graph.outDegrees)

id,outDegree
Doncaster,2
London,2
Den Haag,4
Immingham,2
Amsterdam,3
Colchester,2
Gouda,3
Ipswich,2
Hoek van Holland,3
Felixstowe,2


In [0]:
#What is the most "important" node (has more paths leading to it)
total_degree = graph.degrees
in_degree = graph.inDegrees
out_degree = graph.outDegrees


In [0]:
total_degree.show()

+----------------+------+
|              id|degree|
+----------------+------+
|       Doncaster|     4|
|       Rotterdam|     6|
|          London|     4|
|        Den Haag|     8|
|       Immingham|     4|
|       Amsterdam|     6|
|      Colchester|     4|
|         Utrecht|     4|
|           Gouda|     6|
|         Ipswich|     4|
|Hoek van Holland|     6|
|      Felixstowe|     4|
+----------------+------+



In [0]:
total_degree.join(in_degree, "id", how="left")\
.join(out_degree, "id", how="left")\
.fillna(0)\
.sort("inDegree", ascending=False)\
.show()

+----------------+------+--------+---------+
|              id|degree|inDegree|outDegree|
+----------------+------+--------+---------+
|        Den Haag|     8|       4|        4|
|       Rotterdam|     6|       3|        3|
|       Amsterdam|     6|       3|        3|
|           Gouda|     6|       3|        3|
|Hoek van Holland|     6|       3|        3|
|       Doncaster|     4|       2|        2|
|          London|     4|       2|        2|
|       Immingham|     4|       2|        2|
|      Colchester|     4|       2|        2|
|         Utrecht|     4|       2|        2|
|         Ipswich|     4|       2|        2|
|      Felixstowe|     4|       2|        2|
+----------------+------+--------+---------+



In [0]:
#Performing queries
motifs = graph.find("(Amsterdam)-[e]->(Utrecht)")
display(motifs)

Amsterdam,e,Utrecht
"List(Amsterdam, 52.37919, 4.899431, 821752)","List(Amsterdam, Utrecht, EROAD, 46)","List(Utrecht, 52.092876, 5.10448, 334176)"
"List(Amsterdam, 52.37919, 4.899431, 821752)","List(Amsterdam, Den Haag, EROAD, 59)","List(Den Haag, 52.078663, 4.288788, 514861)"
"List(Den Haag, 52.078663, 4.288788, 514861)","List(Den Haag, Rotterdam, EROAD, 26)","List(Rotterdam, 51.9225, 4.47917, 623652)"
"List(Amsterdam, 52.37919, 4.899431, 821752)","List(Amsterdam, Immingham, EROAD, 369)","List(Immingham, 53.61239, -0.22219, 9642)"
"List(Immingham, 53.61239, -0.22219, 9642)","List(Immingham, Doncaster, EROAD, 74)","List(Doncaster, 53.52285, -1.13116, 302400)"
"List(Doncaster, 53.52285, -1.13116, 302400)","List(Doncaster, London, EROAD, 277)","List(London, 51.509865, -0.118092, 8787892)"
"List(Hoek van Holland, 51.9775, 4.13333, 9382)","List(Hoek van Holland, Den Haag, EROAD, 27)","List(Den Haag, 52.078663, 4.288788, 514861)"
"List(Felixstowe, 51.96375, 1.3511, 23689)","List(Felixstowe, Hoek van Holland, EROAD, 207)","List(Hoek van Holland, 51.9775, 4.13333, 9382)"
"List(Ipswich, 52.05917, 1.15545, 133384)","List(Ipswich, Felixstowe, EROAD, 22)","List(Felixstowe, 51.96375, 1.3511, 23689)"
"List(Colchester, 51.88921, 0.90421, 104390)","List(Colchester, Ipswich, EROAD, 32)","List(Ipswich, 52.05917, 1.15545, 133384)"


In [0]:
#Filtering previous query results
filtered = motifs.filter("e.cost < 30")
display(filtered)

Amsterdam,e,Utrecht
"List(Den Haag, 52.078663, 4.288788, 514861)","List(Den Haag, Rotterdam, EROAD, 26)","List(Rotterdam, 51.9225, 4.47917, 623652)"
"List(Hoek van Holland, 51.9775, 4.13333, 9382)","List(Hoek van Holland, Den Haag, EROAD, 27)","List(Den Haag, 52.078663, 4.288788, 514861)"
"List(Ipswich, 52.05917, 1.15545, 133384)","List(Ipswich, Felixstowe, EROAD, 22)","List(Felixstowe, 51.96375, 1.3511, 23689)"
"List(Gouda, 52.01667, 4.70833, 70939)","List(Gouda, Rotterdam, EROAD, 25)","List(Rotterdam, 51.9225, 4.47917, 623652)"
"List(Rotterdam, 51.9225, 4.47917, 623652)","List(Rotterdam, Den Haag, EROAD, 26)","List(Den Haag, 52.078663, 4.288788, 514861)"
"List(Den Haag, 52.078663, 4.288788, 514861)","List(Den Haag, Hoek van Holland, EROAD, 27)","List(Hoek van Holland, 51.9775, 4.13333, 9382)"
"List(Felixstowe, 51.96375, 1.3511, 23689)","List(Felixstowe, Ipswich, EROAD, 22)","List(Ipswich, 52.05917, 1.15545, 133384)"
"List(Rotterdam, 51.9225, 4.47917, 623652)","List(Rotterdam, Gouda, EROAD, 25)","List(Gouda, 52.01667, 4.70833, 70939)"


In [0]:
#Finding the shortest route between the city of Den Haag and any of the medium sized cities
origin = "id='Den Haag'"
destination = "population > 100000 and population < 300000 and id <> 'Den Haag'"
result = graph.bfs(origin, destination) #BFS finds the shortest path between two nodes

In [0]:
print(result.columns)  #Columns with 'e' represent edges (edges) and columns with 'v' represent nodes (vertices)

['from', 'e0', 'v1', 'e1', 'v2', 'e2', 'to']


In [0]:
#Selected only the nodes who doesn't start with 'e'
columns = [column for column in result.columns if not column.startswith("e")]
result.select(columns).show(5,False)

+---------------------------------------+------------------------------------------+-------------------------------------+------------------------------------+
|from                                   |v1                                        |v2                                   |to                                  |
+---------------------------------------+------------------------------------------+-------------------------------------+------------------------------------+
|{Den Haag, 52.078663, 4.288788, 514861}|{Hoek van Holland, 51.9775, 4.13333, 9382}|{Felixstowe, 51.96375, 1.3511, 23689}|{Ipswich, 52.05917, 1.15545, 133384}|
+---------------------------------------+------------------------------------------+-------------------------------------+------------------------------------+



In [0]:
graph.vertices\
.filter("population > 100000 and population < 300000")\
.sort("population")\
.show()

+----------+--------+---------+----------+
|        id|latitude|longitude|population|
+----------+--------+---------+----------+
|Colchester|51.88921|  0.90421|    104390|
|   Ipswich|52.05917|  1.15545|    133384|
+----------+--------+---------+----------+

